# Effective bounds on H<sub>t</sub>

The following formulae are cut and pasted as LaTeX from [Effective bounds on H_t - second approach](http://michaelnielsen.org/polymath1/index.php?title=Effective_bounds_on_H_t_-_second_approach) (hereafter "the reference"). They are presented in a different order, one which facilitates coding. The object is to implement $E_1$, $E_2$, and $E_3$ as defined in the above link. 

### Previously coded:

The following formulae were coded and integrated previously.

$$H_{0,1}(s) := \frac{s (s-1)}{2} \pi^{-s/2} \sqrt{2\pi} \exp( (\frac{s}{2} - \frac{1}{2}) \log \frac{s}{2} - \frac{s}{2} )$$

$$\alpha_1(s) := \frac{1}{2s} + \frac{1}{s-1} + \frac{1}{2} \log \frac{s}{2\pi}$$

$$T' := \frac{x}{2} + \frac{\pi t}{8}$$

$$N := \lfloor \sqrt{ \frac{T'}{2\pi}} \rfloor$$

### E1, E2, E3

Expressions below are presented here in an order convenient for coding.

By convention, $z=x+iy$ and $s = \frac{1+iz}{2}$. Hence in the expressions below, $\frac{1-y+ix}{2}=s$ and $\frac{1+y+ix}{2} = \overline{1-s}$

The derivation [Effective bounds on H t - second approach](http://michaelnielsen.org/polymath1/index.php?title=Effective_bounds_on_H_t_-_second_approach) also establishes the following conventions, $s = \sigma+iT$, $T \geq T_0 \geq 10$, and $t \leq 1/2$. Thus, $T$ and $T_0$, like $t$ and $s$, are essentially parameters. Two further definitions, $T_0'$ and $\alpha_0$ are simple enough not to require coding as functions.

$$T'_0 := T_0 + \frac{\pi t}{8}$$

$$a_0 := \sqrt{\frac{T'_0}{2\pi}}$$

Since $E_1$ and $E_2$ depend on $\epsilon'(s)$, it's convenient to implement $\epsilon'(s)$ first. Its definition, below, contains terms $Re(s)$ and $Re\alpha_1(s)$ which I take to be the real part of their arguments. That would be consistent with, e.g., the corresponding Python code (function `eps_err`, lines 302-303 of `mputility.py`)

```
    alph = alpha1(s)
    term1 = sigma + (t/2.0) * (alph.real) - (t/4.0) * mp.log(N)
```

$$\epsilon'(s) := \frac{1}{2} \sum_{n=1}^N \frac{1}{n^{\mathrm{Re}(s) + \frac{t \mathrm{Re} \alpha_1(s)}{2} - \frac{t}{4} \log n}}
\exp(\frac{1}{2(T-3.33)} (\frac{t^2}{4} |\alpha_1(s) - \log n|^2 + \frac{1}{3} + t))
(\frac{t^2}{4} |\alpha_1(s) - \log n|^2 + \frac{1}{3} + t )$$

Note that Equation 3.7 of the reference requires $\sigma + \frac{t \mathrm{Re} \alpha_1(s)}{2} - \frac{t}{4} \log N \gt 1$.

In [1]:
using DBNUpperBound
using DBNUpperBound.Asymptotics
using QuadGK

αeff = DBNUpperBound.Asymptotics.αeff # not exported from the package
H01 = DBNUpperBound.Asymptotics.H01   # 

function eps_prime(t::Ty1, s::Ty2, N::Int) where {Ty1<:Real, Ty2<:Number}
    α1 = αeff(s)
    T = imag(s)
    real(s)+t*real(α1-t/4*log(N)) > 1 || error("argument s, ",s," is out of range.")
    ans = 0.0
    for n in 1:N
        α2 = (t^2/4)*abs(α1-log(n))^2 + 1/3 + t
        # For clarity:
        # ans += 1/n^(real(s) + t*real(α1)/2 - t/4*log(n)) * bigexp(α2/(2*(T-3.33))) * α2
        # For numerics:
        ans += bigexp(-n*(real(s) + t*real(α1)/2 - t/4*log(n)) + α2/(2*(T-3.33))) * α2
    end
    return real(ans)/2
end

eps_prime (generic function with 1 method)

$E_1$ can now be implemented.
$$E_1 := \frac{1}{8 (T - 3.33)} \exp( \frac{t}{4} \mathrm{Re} \alpha_1(\frac{1-y+ix}{2})^2 ) |H_{0,1}(\frac{1-y+ix}{2})| \epsilon'(\frac{1-y+ix}{2})$$

In [2]:
function E1(t::Ty1, s::Ty2, N::Int) where {Ty1<:Real, Ty2<:Number}
    T = imag(s)
    return real(1/(8*(T-3.33))*bigexp(t/4*real(αeff(s))^2) * abs(H01(s))*eps_prime(t,s,N))
end

E1 (generic function with 1 method)

Note that the expression for $E_2$ is the same as that for $E_1$ except that $\frac{1+y+ix}{2} = \overline{1-s}$ replaces $\frac{1-y+ix}{2}=s$.

$$E_2 := \frac{1}{8 (T - 3.33)} \exp( \frac{t}{4} \mathrm{Re} \alpha_1(\frac{1+y+ix}{2})^2 ) |H_{0,1}(\frac{1+y+ix}{2})| \epsilon'(\frac{1+y+ix}{2})$$

Thus

In [3]:
function E2(t::Ty1, s::Ty2, N::Int) where {Ty1<:Real, Ty2<:Number}
    return E1(t,(1-s)',N)
end

E2 (generic function with 1 method)

$E3$ depends on $f(\sigma)$, $w(\sigma)$ and $v(\sigma)$. Beginning with
$$f(\sigma) := \frac{1}{2\sqrt{\pi t}} (e^{-(\sigma-(1-y)/2)^2/t} + e^{-(\sigma-(1+y)/2)^2/t}) \quad (4.1)$$

In [4]:
function fσ(t::Ty1, σ::Ty2) where {Ty1<:Real, Ty2<:Real}
    y = 1-σ
    return 1/(2*√(π*t))*(bigexp(-(σ-(1-y)/2)^2/t) + bigexp(-(σ-(1+y)/2)^2)/t)
end

fσ (generic function with 1 method)

$$v(\sigma) := 1 + (0.400 \frac{9^\sigma}{a_0} + 0.346 \frac{2^{3\sigma/2}}{a_0^2}) 1_{\sigma \geq 0} + (9/10)^{\lceil -\sigma \rceil} \sum_{1 \leq k \leq 4-\sigma} (1.1)^k \frac{\Gamma(k/2)}{a_0^k} 1_{\sigma \lt 0}$$

Recall that $T'_0 := T_0 + \frac{\pi t}{8}$, that $a_0 := \sqrt{\frac{T'_0}{2\pi}}$ and that $T_0 \ge 10$.

(Note that $1_{\sigma \geq 0}$ and $1_{\sigma \lt 0}$ represent the characteristic functions of the sets $\sigma \geq 0$ and $\sigma \lt 0$ as per Equations 4.7 and 4.8 of the reference. Note also that $\sigma = 1-y$ where $y$ is the imaginary part of $z$ and is expected to be small, e.g. .4.)

In [5]:
function vσ(t::Ty1, T0::Ty2, σ::Ty3) where {Ty1 <: Real, Ty2 <: Real, Ty3 <:Real}
    T0′ = T0 + π*t/8
    a0 = √(T0′/(2*π))
    σ = real(s)
    if σ ≥ 0
        return 1 + (0.400*9^σ/a0 + 0.346*2^(3σ/2)/a0^2)
    else
        psum = 0.0
        for k in 1:1:floor(Int,(4-σ))
            psum += (1.1)^k*bigΓ(k/2)/a0^k
        end
        return real((9/10)^ceiling(Int,-σ)*psum)
    end
end

vσ (generic function with 1 method)

$$w(\sigma) := (1 + \frac{\sigma^2}{(T'_0)^2})^{1/2} (1 + \frac{(1-\sigma)^2}{(T'_0)^2})^{1/2} 
\exp( \frac{(\sigma-1)_+}{4} \log (1 + \frac{\sigma^2}{(T'_0)^2}) + (\frac{T'_0}{2} \arctan \frac{\sigma}{T'_0} - \frac{\sigma}{2}) 1_{\sigma \lt 0} + \frac{1}{12(T'_0 - 0.33)})$$

In [6]:
function wσ(t::Ty1, T0::Ty2, σ::Ty3) where {Ty1 <: Real, Ty2 <: Real, Ty3 <:Real}
    T0′ = T0 + π*t/8
    a0 = √(T0′/(2*π))
    xpo = max(0,(σ-1))/4*log(1 + σ^2/T0′^2) + (σ<0 ? T0′/2*atan(σ/T0′) : 0) + 1/(12*(T0′-0.33))
    return real((1+σ^2/T0′^2)^(1/2)*(1 + (1-σ)^2/(T0′^2))^(1/2)*bigexp(xpo))
end

wσ (generic function with 1 method)

$$E_3 := \frac{1}{8} \sqrt{\pi} \exp( -\frac{t \pi^2}{64} ) (T')^{3/2} e^{-\pi T/4}  \int_{-\infty}^\infty v(\sigma) w(\sigma) f(\sigma)\ d\sigma$$

where $T' := \frac{x}{2} + \frac{\pi t}{8}$.

In [7]:
function E3(t::Ty1, T0::Ty1, s::Ty2) where {Ty1 <: Real, Ty2 <: Number}
    T0 ≥ 10 || error("T0 = ",T0," is out of applicable range.")
    T = imag(s)
    T ≥ T0 || error("T = ", T, "must be at least ", T0, " (T0).")
    Tprime = imag(s) + π*t/8
    t ≤ 1/2 || error("t must not exceed 1/2. Given value is ",t,".")
    σ = real(s)
    # sum quadgk's estimate of the integral and the error in its estimate
    defint = quadgk((σ)->vσ(t,T0,σ)*wσ(t,T0,σ)*fσ(t,σ),-Inf,-10,10,Inf,abstol=eps(BigFloat))
    μ = real(1/8*√(π)bigexp(-t*π^2/64-π*T/4)*Tprime^3/2)
    return μ*real(defint[1]),μ*defint[2]
end


E3 (generic function with 1 method)

## Test run

In [8]:
z = 1e6 + .4*im
s = (1+im*z)/2
t = .4
N = Ns(s)
@show z
@show s
@show t
@show N
nothing

z = 1.0e6 + 0.4im
s = 0.3 + 500000.0im
t = 0.4
N = 282


In [9]:
@show E1(t,s,N)
@show E2(t,s,N)
@show E3(t,10.0,s)
nothing

E1(t, s, N) = 5.974151239790172913977033261736278935303608336105512272609633006544968619583181e-170544
E2(t, s, N) = 3.449080910233385473528960988565769170175109715726004360937464078504143428365288e-170543
E3(t, 10.0, s) = (1.690412513800519271553799943585169448019823781866850997043673977336102224622558e-85257, 2.028831860207788805401245931938005045677672444228430169808973111179036739524028e-85265)


E3, which involves an approximate integral, is suspect even though the estimated integration error is several orders of magnitude smaller than the estimated value.